In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from collections import Counter

data = pd.read_csv('Binary_data.csv')
# data = pd.read_csv('/Users/macbookair/workspace/COM804/Approach_1/Multi_data.csv')

### Binary Pred

In [2]:
X = data.iloc[:,:-1]
y = data.iloc[:, -1]

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3, random_state=31)
X_val, X_test, y_val, y_test = train_test_split(X_train, y_train, test_size= 0.5, random_state=31)

In [4]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [5]:
def normalize_features(X, X_train):
    
    eps = 1e-12
    X_min = X_train.min(axis=0)
    X_max = X_train.max(axis=0)
    diff  = np.maximum(X_max - X_min, eps)
    X_norm = (X - X_min) / diff
    
    return X_norm

X_train_norm = normalize_features(X_train, X_train)
X_val_norm = normalize_features(X_val, X_train)

In [6]:
X_train_norm = normalize_features(X_train, X_train)
X_val_norm = normalize_features(X_val, X_train)


In [7]:
X_train_norm.head()

,Sex,Age,Systolic_Pressure,BMI,CKD_Cause,Hemoglobin,Albumin,Creatinine,eGFR,CKD_Stage,CKD_Risk,Dipstick_Proteinuria,Proteinuria,Occult_Blood_in_Urine,Protein_Creatinine_Ratio,UPCR_Severity,Hypertension,Previous_CVD,Diabetes,RAAS_Inhibitor,Calcium_Channel_Blocker,Diuretics
0,0.0,0.000000,0.223684,0.237548,0.666667,0.731092,0.864865,0.091116,1.000000,0.000000,0.090909,0.4,1.0,0.0,0.009415,0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.861111,0.355263,0.291188,0.333333,0.521008,0.837838,0.476082,0.077676,1.000000,1.000000,0.0,0.0,0.0,0.025273,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.611111,0.368421,0.241379,0.333333,0.915966,0.702703,0.148064,0.516477,0.333333,0.363636,0.2,0.0,0.0,0.018335,0.5,1.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.750000,0.250000,0.425287,0.333333,0.487395,0.648649,0.248292,0.317270,0.333333,0.363636,0.0,0.0,1.0,0.009911,0.5,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.708333,0.467105,0.517241,0.333333,0.537815,0.648649,0.179954,0.429138,0.333333,0.272727,0.0,0.0,1.0,0.000991,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [8]:
num_cols = ['Age', 'Systolic_Pressure', 'BMI', 'Hemoglobin', 'Albumin', 'Creatinine', 'eGFR', 'Protein_Creatinine_Ratio']
cat_cols = [c for c in X_train if c not in num_cols]

In [9]:
### printt the number of unique values in each feature
for i in X_train.columns:
    print(f"{i}: {X_train[i].nunique()}")

Sex: 2
Age: 66
Systolic_Pressure: 107
BMI: 169
CKD_Cause: 4
Hemoglobin: 107
Albumin: 36
Creatinine: 274
eGFR: 626
CKD_Stage: 4
CKD_Risk: 12
Dipstick_Proteinuria: 6
Proteinuria: 2
Occult_Blood_in_Urine: 2
Protein_Creatinine_Ratio: 315
UPCR_Severity: 3
Hypertension: 2
Previous_CVD: 2
Diabetes: 2
RAAS_Inhibitor: 2
Calcium_Channel_Blocker: 2
Diuretics: 2


In [10]:
## seperating the numerical columns and the categorical columns
cat_cols = [col for col in X_train.columns if X_train[col].nunique() <= 12]
num_cols = [c for c in X_train if c not in cat_cols]

In [11]:
# # Initialize the weights to 1
# W = {}
# for col in X_train.columns:
#     W[col] = 1.0
n = X_train_norm.shape[1]
W = np.ones(n,)

In [12]:
### similarity search
def local_sims(new_case, casebase, num_cols, cat_cols, eps=1e-12):
    sims = {}
    
    for col in num_cols:
            diff = np.abs(new_case[col] - casebase[col])
            f_min = casebase[col].min()
            f_max = casebase[col].max()
            f_range = np.maximum(f_max - f_min, eps)
            sims[col] = (1 - (diff / f_range)).clip(0, 1)
        
    for col in cat_cols:
            sims[col] = (casebase[col] == new_case[col]).astype(float)
    
    df_sim = pd.DataFrame(sims, index=casebase.index)
    ordered_cols = [c for c in casebase.columns if c in df_sim.columns]
    
    return df_sim[ordered_cols]

### gloabl similarity search

def global_sims(X, W, top_k = 5):
    W = np.asarray(W, dtype=float)
    X = X.to_numpy(dtype=float)
    
    gs = (X * W).sum(axis=1) / W.sum()
    sorted_gs = np.argsort(gs)[::-1][:top_k]          ## returns the top 5 rows of the most similar cases
    # best_idx = np.argmax(gs)
    
    return sorted_gs, gs


# retrieve the similar codes with the actual
def retrieval(idx, casebase, y_train): 
    retrieved_x = casebase.iloc[idx].copy()
    retrieved_y = y_train.iloc[idx]
    retrieved_x['CKD Progression'] = retrieved_y.to_numpy()
    
    return retrieved_x, retrieved_y

## make prediction of the result
# def reuse(idx, casebase, y_train):
#     _, y_retrieve = retrieval(idx, casebase, y_train)
def reuse(retrieved_y):
    yhat = np.bincount(retrieved_y).argmax()
    
    return np.array(yhat)



# def evaluation(pred, actual):
#     missclassified = 0
#     for i in range(len(pred)):
#         if pred[i] != actual[i]:
#             missclassified += 1
    
#     fraction_error = missclassified/len(pred)
    
#     return missclassified, fraction_error
    
def evaluation(pred, actual):
        pred = np.asarray(pred)
        actual = np.asarray(actual)
        misclassified = (pred != actual).sum()
        fraction_error = misclassified / len(actual)
        
        return misclassified, fraction_error

In [19]:
new_case = X_val_norm.iloc[0,:]


In [21]:
idx, values = global_sims((local_sims(new_case, X_train_norm, num_cols, cat_cols)), W)

In [14]:
### implementation


def model(new_case, y_val, X_train_norm, y_train, num_cols, cat_cols, W, top_k=5):
    predictions = []
    
    for _, case in new_case.iterrows():
        l_sims = local_sims(case, X_train_norm, num_cols, cat_cols)
        idx, gs = global_sims(l_sims, W, top_k)
        _, retrieved_y = retrieval(idx, X_train_norm, y_train)
        
        solution = reuse(retrieved_y)
        predictions.append(solution)
    
    ## Evaluation
    missclassified, fraction_error  = evaluation(predictions, y_val)
    
    return missclassified, fraction_error

In [15]:
misclassified, fraction_error = model(X_val, y_val, X_train_norm, y_train, num_cols, cat_cols, W, top_k=5)
print(misclassified)
print(fraction_error)

87
0.25663716814159293


### GRADIENT DESCENT

In [23]:
def softamx(z):
    ez = np.exp(z)
    sm = ez / np.sum(ez)
    
    return(sm)
    

In [33]:
### compute cost 

def compute_gradient_cost(X, y_val, W, X_train_norm, num_cols, cat_cols, top_k=5):
    m, n = X.shape
    dj_dw = np.zeros((n,))
    y_val = np.asarray(y_val)
    
    for i, case in X.iterrows():
        l_s_i = local_sims(case, X_train_norm, num_cols, cat_cols)
        idx, Z_i = global_sims(l_s_i, W, top_k)
        f_wb_i = softamx(Z_i)
        
        err_i = f_wb_i - y_val[i]
        
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i, j]
    
    dj_dw = dj_dw/m
    
    return dj_dw

In [34]:
compute_gradient_cost(X_val, y_val, W, X_train_norm, num_cols, cat_cols, top_k=5)

KeyError: (0, 0)